# JOUR 3 — SQL : Jointures et agrégations
## Objectif : apprendre à combiner plusieurs tables et à faire des calculs agrégés


In [3]:
import pandas as pd
import sqlite3

# Charger le dataset Titanic
df = pd.read_csv("../data/train.csv")

# Connexion à la base SQLite
conn = sqlite3.connect("../data/titanic.db")

# Créer ou remplacer la table passengers
df.to_sql("passengers", conn, if_exists="replace", index=False)



891

In [4]:
# Créer un DataFrame ticket fictif
tickets = pd.DataFrame({
    "Ticket": df["Ticket"],
    "Fare_category": ["low" if x < 20 else "medium" if x < 50 else "high" for x in df["Fare"]]
})

# Ajouter dans SQLite
tickets.to_sql("tickets", conn, if_exists="replace", index=False)

# Vérifier
pd.read_sql("SELECT * FROM tickets LIMIT 5;", conn)


,Ticket,Fare_category
0,A/5 21171,low
1,PC 17599,high
2,STON/O2. 3101282,low
3,113803,high
4,373450,low


La cellule 2 crée une table supplémentaire tickets pour que tu puisses ensuite pratiquer les jointures SQL avec la table passengers.

Elle ne modifie pas le dataset Titanic original.

Elle sert uniquement à s’entraîner à INNER JOIN / LEFT JOIN et à l’agrégation.

1. Création un DataFrame pandas tickets :
    - df["Ticket"] → récupère la colonne Ticket du dataset Titanic.

    - ["low" if x < 20 else "medium" if x < 50 else "high" for x in df["Fare"]] → crée une nouvelle colonne Fare_category qui classe le prix du billet (Fare) en trois catégories :

        - low si < 20

        - medium si >=20 et <50

        - high si >=50

    - Résultat : un nouveau DataFrame tickets avec deux colonnes : Ticket | Fare_category

2. Écriture de la table dans SQLite
    - to_sql → envoie le DataFrame tickets dans la base SQLite connectée via conn.

    - "tickets" → nom de la table créée.

    - if_exists="replace" → si la table existe déjà, elle est remplacée.

    - index=False → ne pas inclure l’index du DataFrame comme colonne dans la table.

3. Vérifier le contenu
    - Exécute une requête SQL pour afficher les 5 premières lignes de la table tickets.

    - Permet de vérifier que la table a bien été créée et que les colonnes sont correctes.

In [5]:
# 1. INNER JOIN : combiner passengers avec tickets
query = """
SELECT p.Name, p.Sex, p.Age, t.Fare_category
FROM passengers p
INNER JOIN tickets t
ON p.Ticket = t.Ticket
LIMIT 10;
"""
pd.read_sql(query, conn)

,Name,Sex,Age,Fare_category
0,"Braund, Mr. Owen Harris",male,22.0,low
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,high
2,"Heikkinen, Miss. Laina",female,26.0,low
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,high
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,high
5,"Allen, Mr. William Henry",male,35.0,low
6,"Moran, Mr. James",male,NaN,low
7,"McCarthy, Mr. Timothy J",male,54.0,high
8,"Palsson, Master. Gosta Leonard",male,2.0,medium
9,"Palsson, Master. Gosta Leonard",male,2.0,medium


L’objectif : associer chaque passager à sa catégorie de tarif en utilisant le numéro de ticket.

1. SELECT p.Name, p.Sex, p.Age, t.Fare_category

    - On choisit les colonnes à afficher.

    - p. = alias pour la table passengers

    - t. = alias pour la table tickets

2. FROM passengers p
    - On commence avec la table passengers et on lui donne l’alias p pour simplifier les références.

3. INNER JOIN tickets t
    - INNER JOIN combine deux tables.
    - Ici on joint passengers (p) avec tickets (t).

4. ON p.Ticket = t.Ticket
    - Condition de jointure : on relie les deux tables lorsque le numéro de ticket est identique.
    - Seules les lignes qui ont un ticket correspondant dans les deux tables seront conservées.
5. LIMIT 10
    - Affiche uniquement les 10 premières lignes pour vérifier que ça fonctionne.

Le résultat est un DataFrame pandas avec ces colonnes :
- Chaque passager est associé à sa catégorie de tarif (Fare_category) via son ticket.

- Si un passager n’a pas de ticket correspondant dans tickets, il n’apparaît pas dans l’INNER JOIN.

In [ ]:
# Filtre par Fare_category='high' pour ne voir que les passagers avec des billets chers.
query = """
SELECT p.Name, p.Sex, p.Age, t.Fare_category
FROM passengers p
INNER JOIN tickets t
ON p.Ticket = t.Ticket and Fare_category='high'
LIMIT 10;
"""
pd.read_sql(query, conn)

,Name,Sex,Age,Fare_category
0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,high
1,"Futrelle, Mr. Jacques Heath",male,37.0,high
2,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,high
3,"McCarthy, Mr. Timothy J",male,54.0,high
4,"Fortune, Miss. Alice Elizabeth",female,24.0,high
5,"Fortune, Miss. Mabel Helen",female,23.0,high
6,"Fortune, Mr. Charles Alexander",male,19.0,high
7,"Fortune, Mr. Mark",male,64.0,high
8,"Lurette, Miss. Elise",female,58.0,high
9,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,high


In [12]:
# 2. LEFT JOIN : tous les passagers, même sans ticket
query = """
SELECT p.Name, p.Sex, t.Fare_category
FROM passengers p
LEFT JOIN tickets t
ON p.Ticket = t.Ticket
LIMIT 10;
"""
pd.read_sql(query, conn)

,Name,Sex,Fare_category
0,"Braund, Mr. Owen Harris",male,low
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,high
2,"Heikkinen, Miss. Laina",female,low
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,high
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,high
5,"Allen, Mr. William Henry",male,low
6,"Moran, Mr. James",male,low
7,"McCarthy, Mr. Timothy J",male,high
8,"Palsson, Master. Gosta Leonard",male,medium
9,"Palsson, Master. Gosta Leonard",male,medium


1. SELECT p.Name, p.Sex, t.Fare_category

- On sélectionne :

    - Name et Sex depuis passengers (p)

    - Fare_category depuis tickets (t)

2. FROM passengers p

- Table de départ : passengers

- Alias p pour simplifier les références

3. LEFT JOIN tickets t

- Joint passengers avec tickets en gardant tous les passagers, même si leur ticket n’existe pas dans tickets.

4. ON p.Ticket = t.Ticket

- Condition de correspondance : Ticket doit être identique pour combiner les informations.

5. LIMIT 10

- Affiche uniquement les 10 premières lignes pour tester

### Résultat attendu

- NULL apparaît pour Fare_category si le passager n’a pas de ticket correspondant dans tickets.

- Tous les passagers de la table passengers apparaissent, contrairement à INNER JOIN.

In [ ]:
# Compter combien de passagers n'ont pas de ticktet
query = """
SELECT COUNT(*) FROM passengers p
LEFT JOIN tickets t
ON p.Ticket = t.Ticket
WHERE t.Fare_category IS NULL;
"""
pd.read_sql(query,conn)

,COUNT(*)
0,0


In [ ]:
# 1. Nombre de survivants par classe
query = """
SELECT Pclass, COUNT(*) as Survivors
FROM passengers
WHERE Survived = 1
GROUP BY Pclass;
"""
pd.read_sql(query, conn)

In [ ]:
# 2. Age moyen par sexe
query = """
SELECT Sex, AVG(Age) as Average_Age
FROM passengers
GROUP BY Sex;
"""

In [ ]:
# 3. Taux de survie par classe
query = """
SELECT Pclass,
       SUM(Survived)*1.0/COUNT(*) as Survival_Rate
FROM passengers
GROUP BY Pclass;
"""
pd.read_sql(query, conn)